In [13]:
import numpy as np
import pandas as pd
import torch
from PIL import Image
from torchvision import transforms
import os
import matplotlib.pyplot as plt
import ast
from functools import partial

# First attempt: Use WordNet similarity score to find probabilitiy corresponding to the closest label 

In [14]:
from nltk.corpus import wordnet

In [15]:
# df = pd.read_csv('alexnet_scores_wordnet_id.csv',index_col=0)

Input dataframe columns: (see alexnet.ipynb for getting wordnet ids for corresponding labels)

In [19]:
df.columns

Index(['type', 'dir', 'id_labels', 'readable_labels'], dtype='object')

In [20]:
# manually select the synsnet with the correct definition for each type
type_names = np.unique(df['type'].values)
for t in type_names:
    syns = wordnet.synsets(t.lower())
    if t == 'Beach':
        syns = wordnet.synsets('seashore')
    print(f'{t}: ')
    for s in syns:
        print(s)
        print(s.definition())
    print()

Banana: 
Synset('banana.n.01')
any of several tropical and subtropical treelike herbs of the genus Musa having a terminal crown of large entire leaves and usually bearing hanging clusters of elongated fruits
Synset('banana.n.02')
elongated crescent-shaped yellow fruit with soft sweet flesh

Beach: 
Synset('seashore.n.01')
the shore of a sea or ocean

Car: 
Synset('car.n.01')
a motor vehicle with four wheels; usually propelled by an internal combustion engine
Synset('car.n.02')
a wheeled vehicle adapted to the rails of railroad
Synset('car.n.03')
the compartment that is suspended from an airship and that carries personnel and the cargo and the power plant
Synset('car.n.04')
where passengers ride up and down
Synset('cable_car.n.01')
a conveyance for passengers or freight on a cable railway

Church: 
Synset('church.n.01')
one of the groups of Christians who have their own beliefs and forms of worship
Synset('church.n.02')
a place for public (especially Christian) worship
Synset('church_se

In [21]:
type_synsets = ['banana.n.02','seashore.n.01','car.n.01','church.n.02','beacon.n.03','mountain.n.01','coffee_mug.n.01','airplane.n.01']
type_final = {type_names[i] : type_synsets[i] for i in range(len(type_names))}

In [43]:
type_final

{'Banana': 'banana.n.02',
 'Beach': 'seashore.n.01',
 'Car': 'car.n.01',
 'Church': 'church.n.02',
 'Lighthouse': 'beacon.n.03',
 'Mountain': 'mountain.n.01',
 'Mug': 'coffee_mug.n.01',
 'Plane': 'airplane.n.01'}

In [10]:
# read labels 
with open("../imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# read labels to wordnet synsets
ltw = pd.read_json('../imagenet_label_to_wordnet_synset.json').T

In [21]:
# compute similarity score between synsets of each of the top 10 labels and the type 
def getSyns(similarity,closest,row):
    # type name
    category = row['type'] 
    
    # top 10 labels' ids in wordnet
    labels  = row['id_labels'] 
    labels = ast.literal_eval(labels)
    key = labels.keys() 
    
    # iterate through top 10 labels and compare with type 
    sim = dict()
    notFound = True
    for k in key:
        pos, synid = k.split('-')[1], int(k.split('-')[0]) # part of speech (all noun in this case), id
        k_syns = wordnet.synset_from_pos_and_offset(pos, synid) # label synsets
        cat_syns = wordnet.synset(type_final[category]) # type synsets
        s = k_syns.path_similarity(cat_syns)
        sim[k] = s
        
        # find the first > 0.5 similarity label among the top 10
        if s >= 0.5 and notFound:
            closest.append({k:s})
            notFound = False
            
    # return the label with highest probabilitiy if none is above 0.5
    if notFound:
        sim_np = np.array(sim.values())
        max_key = list(key)[np.argmax(sim_np)]
        closest.append({max_key : sim[max_key]})
    
    similarity.append(sim)

In [23]:
# get readable labels from imagenet_label_to_wordnet_synset.json
def convertIdToReadable(id_label):
    for idx in range(ltw.shape[0]):
        if ltw['id'][idx] == id_label:
            return ltw['label'][idx]
    return None

In [24]:
df

,type,dir,id_labels,readable_labels
0,Mountain,../Image/All_Cropped\Mountain\1.jpg,"{'09193705-n': 0.1344020962715149, '09472597-n...","{'alp': 0.1344020962715149, 'volcano': 0.13413..."
1,Mountain,../Image/All_Cropped\Mountain\10.jpg,"{'09468604-n': 0.4578712582588196, '09193705-n...","{'valley': 0.4578712582588196, 'alp': 0.168182..."
2,Mountain,../Image/All_Cropped\Mountain\11.jpg,"{'09472597-n': 0.6781872510910034, '09193705-n...","{'volcano': 0.6781872510910034, 'alp': 0.11668..."
3,Mountain,../Image/All_Cropped\Mountain\12.jpg,"{'09193705-n': 0.5507320761680603, '09468604-n...","{'alp': 0.5507320761680603, 'valley': 0.412458..."
4,Mountain,../Image/All_Cropped\Mountain\13.jpg,"{'09246464-n': 0.4606900215148926, '09399592-n...","{'cliff': 0.4606900215148926, 'promontory': 0...."
...,...,...,...,...
123,Church,../Image/All_Cropped\Church\5.jpg,"{'03028079-n': 0.8265923261642456, '02825657-n...","{'church': 0.8265923261642456, 'bell cote': 0...."
124,Church,../Image/All_Cropped\Church\6.jpg,"{'02793495-n': 0.8163257241249084, '03028079-n...","{'barn': 0.8163257241249084, 'church': 0.06418..."
125,Church,../Image/All_Cropped\Church\7.jpg,"{'03781244-n': 0.3951680362224579, '03028079-n...","{'monastery': 0.3951680362224579, 'church': 0...."
126,Church,../Image/All_Cropped\Church\8.jpg,"{'03028079-n': 0.8428629040718079, '03930313-n...","{'church': 0.8428629040718079, 'picket fence':..."


In [25]:
similarity = []
closest = []
getSynsPartial = partial(getSyns, similarity, closest)
df.apply(getSynsPartial, axis=1)

df['similarity_score'] = similarity
df['closest_id_label'] = [list(item.keys())[0] for item in closest]
df['closest_similarity_score'] = [list(item.values())[0] for item in closest]
df['closest_readable_label'] = df['closest_id_label'].apply(convertIdToReadable)
df['closest_prob'] = [ast.literal_eval(d)[i] for d, i in zip(df['id_labels'],df['closest_id_label'])]

In [26]:
df

,type,dir,id_labels,readable_labels,similarity_score,closest_id_label,closest_similarity_score,closest_readable_label,closest_prob
0,Mountain,../Image/All_Cropped\Mountain\1.jpg,"{'09193705-n': 0.1344020962715149, '09472597-n...","{'alp': 0.1344020962715149, 'volcano': 0.13413...","{'09193705-n': 0.5, '09472597-n': 0.5, '043100...",09193705-n,0.5,alp,0.134402
1,Mountain,../Image/All_Cropped\Mountain\10.jpg,"{'09468604-n': 0.4578712582588196, '09193705-n...","{'valley': 0.4578712582588196, 'alp': 0.168182...","{'09468604-n': 0.2, '09193705-n': 0.5, '094725...",09193705-n,0.5,alp,0.168182
2,Mountain,../Image/All_Cropped\Mountain\11.jpg,"{'09472597-n': 0.6781872510910034, '09193705-n...","{'volcano': 0.6781872510910034, 'alp': 0.11668...","{'09472597-n': 0.5, '09193705-n': 0.5, '094686...",09472597-n,0.5,volcano,0.678187
3,Mountain,../Image/All_Cropped\Mountain\12.jpg,"{'09193705-n': 0.5507320761680603, '09468604-n...","{'alp': 0.5507320761680603, 'valley': 0.412458...","{'09193705-n': 0.5, '09468604-n': 0.2, '093328...",09193705-n,0.5,alp,0.550732
4,Mountain,../Image/All_Cropped\Mountain\13.jpg,"{'09246464-n': 0.4606900215148926, '09399592-n...","{'cliff': 0.4606900215148926, 'promontory': 0....","{'09246464-n': 0.25, '09399592-n': 0.333333333...",09193705-n,0.5,alp,0.029514
...,...,...,...,...,...,...,...,...,...
123,Church,../Image/All_Cropped\Church\5.jpg,"{'03028079-n': 0.8265923261642456, '02825657-n...","{'church': 0.8265923261642456, 'bell cote': 0....","{'03028079-n': 1.0, '02825657-n': 0.1111111111...",03028079-n,1.0,"church, church building",0.826592
124,Church,../Image/All_Cropped\Church\6.jpg,"{'02793495-n': 0.8163257241249084, '03028079-n...","{'barn': 0.8163257241249084, 'church': 0.06418...","{'02793495-n': 0.2, '03028079-n': 1.0, '044176...",03028079-n,1.0,"church, church building",0.064184
125,Church,../Image/All_Cropped\Church\7.jpg,"{'03781244-n': 0.3951680362224579, '03028079-n...","{'monastery': 0.3951680362224579, 'church': 0....","{'03781244-n': 0.14285714285714285, '03028079-...",03028079-n,1.0,"church, church building",0.183275
126,Church,../Image/All_Cropped\Church\8.jpg,"{'03028079-n': 0.8428629040718079, '03930313-n...","{'church': 0.8428629040718079, 'picket fence':...","{'03028079-n': 1.0, '03930313-n': 0.125, '0279...",03028079-n,1.0,"church, church building",0.842863


In [29]:
# df.to_csv('alexnet_scores_wordnet_id_closest_label.csv')

Potential issues: 
1. top 10 and threshold of 0.5 are arbitrarily chosen - e.g. see `df.loc[4]` where the top label is `cliff` (similarity score 0.25) but `alp` is the chosen label
2. when none of top 10 labels' similarity scores pass the 0.5 threshold, it chooses the label with max similarity score and breaks ties by choosing the first, which is one with highest probability among all max similarity score labels - e.g. see `df.loc[9]`


In [27]:
df.loc[4]

type                                                                 Mountain
dir                                      ../Image/All_Cropped\Mountain\13.jpg
id_labels                   {'09246464-n': 0.4606900215148926, '09399592-n...
readable_labels             {'cliff': 0.4606900215148926, 'promontory': 0....
similarity_score            {'09246464-n': 0.25, '09399592-n': 0.333333333...
closest_id_label                                                   09193705-n
closest_similarity_score                                                  0.5
closest_readable_label                                                    alp
closest_prob                                                        0.0295136
Name: 4, dtype: object

In [28]:
df.loc[9]

type                                                                 Mountain
dir                                       ../Image/All_Cropped\Mountain\3.jpg
id_labels                   {'09288635-n': 0.7891390323638916, '04604644-n...
readable_labels             {'geyser': 0.7891390323638916, 'worm fence': 0...
similarity_score            {'09288635-n': 0.2, '04604644-n': 0.0833333333...
closest_id_label                                                   09193705-n
closest_similarity_score                                                  0.5
closest_readable_label                                                    alp
closest_prob                                                        0.0164292
Name: 9, dtype: object

# Second attempt: Map all labels using similarity score

In [60]:
# read labels 
with open("../imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# read labels to wordnet synsets
ltw = pd.read_json('../imagenet_label_to_wordnet_synset.json').T

In [61]:
for t in type_final:
    sim = []
    for i in range(ltw.shape[0]):
        type_synset = wordnet.synset(type_final[t])
        k = ltw.iloc[i]['id']
        pos, synid = k.split('-')[1], int(k.split('-')[0]) # part of speech (all noun in this case), id
        k_syns = wordnet.synset_from_pos_and_offset(pos, synid) # label synsets
        sim.append(k_syns.path_similarity(type_synset))
    ltw[t] = sim

In [62]:
type_labels = dict()
for t in type_final:
    if t in ['Plane']:
        temp = np.array(ltw[ltw[t] >= 0.3].sort_values(t, ascending=False).index)
    else:
        temp = np.array(ltw[ltw[t] >= 0.5].sort_values(t, ascending=False).index)
    if temp.size == 1:
        type_labels[t] = temp[0]
        print(f"{t}: {ltw.loc[type_labels[t]]['label']}")
        print(f"{t}: {ltw.loc[type_labels[t]][t]}")
        print()
    else:
        type_labels[t] = temp
        print(f"{t}: {ltw.loc[type_labels[t]]['label'].values}")
        print(f"{t}: {ltw.loc[type_labels[t]][t].values}")
        print()

Banana: banana
Banana: 1.0

Beach: seashore, coast, seacoast, sea-coast
Beach: 1.0

Car: ['ambulance'
 'beach wagon, station wagon, wagon, estate car, beach waggon, station waggon, waggon'
 'cab, hack, taxi, taxicab' 'convertible' 'jeep, landrover'
 'limousine, limo' 'minivan' 'Model T' 'racer, race car, racing car'
 'sports car, sport car']
Car: [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]

Church: church, church building
Church: 1.0

Lighthouse: beacon, lighthouse, beacon light, pharos
Lighthouse: 1.0

Mountain: ['alp' 'volcano']
Mountain: [0.5 0.5]

Mug: coffee mug
Mug: 1.0

Plane: ['airliner' 'warplane, military plane']
Plane: [0.5        0.33333333]



In [67]:
# Beach(seashore), Banana, Mug(coffee mug), Church, Lighthouse(beacon) have corresponding labels in the imagenet labels
# Car, Mountain, Plane don't have corresponding labels
# 'beach wagon, station wagon, wagon, estate car, beach waggon, station waggon, waggon' 436
# 'alp' 970
# 'airliner' 404

In [68]:
# df = pd.read_csv('alexnet_scores_wordnet_id_all_prob.csv',index_col=0)

In [69]:
# def getScoreForMappedLabels(matched_label,matched_prob,row):
#     top_labels = ast.literal_eval(row['id_labels'])
#     mapped_labels = ltw.loc[type_labels[row['type']]]['id'].values
#     found =False
#     for k in top_labels.keys():
#         for l in mapped_labels:
#             if k == l:
#                 matched_label.append(k)
#                 matched_prob.append(top_labels[k])
#                 found = True
#                 break
#         if found:
#             break
#     if not found:
#         print('not found')
#         matched_label.append(None)
#         matched_prob.append(None)

In [70]:
# matched_label,matched_prob = [], []
# getScoreForMappedLabelsPartial = partial(getScoreForMappedLabels, matched_label,matched_prob)
# df.apply(getScoreForMappedLabelsPartial,axis=1)

# df['matched_label'] = matched_label
# df['matched_prob'] = matched_prob
# df['matched_readable'] = df['matched_label'].apply(lambda x: ltw[ltw['id'] == x]['label'].values[0])

In [120]:
# df.to_csv('alexnet_scores_wordnet_id_matched_label.csv')

# Final label selection: select one label that most closely matches our definitions of the categories

Select one label that's closest to our definition of the categories

In [80]:
type_labels['Car'] = 436
type_labels['Mountain'] = 970
type_labels['Plane'] = 404

In [58]:
# ltw.loc[list(type_labels.values())][ltw.columns[:3]].to_csv('selected_labels.csv')

In [81]:
df = pd.read_csv('alexnet_scores_wordnet_id_all_prob.csv',index_col=0)

In [87]:
def getScoreForMappedLabels(matched_label,matched_prob,row):
    top_labels = ast.literal_eval(row['id_labels'])
    mapped_label = ltw.loc[type_labels[row['type']]]['id']
    for k in top_labels.keys():
        if k == mapped_label:
            matched_label.append(k)
            matched_prob.append(top_labels[k])

In [88]:
matched_label,matched_prob = [], []
getScoreForMappedLabelsPartial = partial(getScoreForMappedLabels, matched_label,matched_prob)
df.apply(getScoreForMappedLabelsPartial,axis=1)

df['matched_label'] = matched_label
df['matched_prob'] = matched_prob
df['matched_readable'] = df['matched_label'].apply(lambda x: ltw[ltw['id'] == x]['label'].values[0])

In [90]:
# df.to_csv('alexnet_scores_wordnet_id_matched_label.csv')

In [59]:
# import matplotlib.pyplot as plt

# for t in type_names:
#     df_plot=df[df['type']==t]
#     plt.hist(df_plot['closest_prob'])
#     plt.title(t)
#     plt.show()